# 自定义数据集建立
 数据集有可以直接从网上下载的，但大多数数据集实际上还需要手动处理，按照pytorch训练的格式写进去

In [1]:
import torch
from torch.utils.data import Dataset
class Self_Dataset(Dataset):
    def __init__(self,datas,lables):
        self.datas = datas
        self.lables = lables

    def __len__(self):
        return len(self.lables)

    def __getitem__(self, item):
        data=self.datas[item]
        label=self.labels[item]
        return data,label

  这基本上是一个最基础的框架，实际上要运用的会比这个要复杂，但是只要记住，自定义数据需要继承torch.utils.data中的Dataset类，并且重写三个函数即可，重点是第三个函数，他决定了数据集返回什么，一般是输入数据，加标签。
  ### 标签
  标签部分只用写一个下标i，表示对应第i个分类结果是正确答案，在损失函数内部会自动处理为独热编码

### 模拟建立一个自己的数据集
这里选择创建100组数据，每一组数据有两个值(x,y),代表在空间坐标系的坐标，现在我们想训练把数据分为两类，一类在y=x^2方函数的上方，另一组在下方，所以就要对它们贴上不同的标签，上方标签为0，下方标签为1

In [10]:
datas=torch.rand(1000,2)
#这里是贴标签的函数
def get_label(data):
    if data[0]*data[0] - data[1] >=0:
        return 0
    else:
        return 1

labels=torch.tensor([get_label(data)for data in datas ])
class XY_Dataset(Dataset):
    def __init__(self,datas,labels):
        self.datas = datas
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        data=self.datas[item]
        label=self.labels[item]
        return data,label

xy_dataset=XY_Dataset(datas,labels)
print(xy_dataset[0])


(tensor([0.2787, 0.0385]), tensor(0))


### 划分训练集和测试集
这一步并不是每次都需要，但有些时候拿到的数据并不会区分训练集和测试集，这时候就需要手动划分

In [11]:
p=0.9
train_size=int(len(xy_dataset)*p)
test_size=len(xy_dataset)-train_size

注意，这里需要把结果转为整数先，因为总数*0.9可能是小数

In [17]:
train_dataset,test_dataset=torch.utils.data.random_split(xy_dataset,[train_size,test_size])
print(f"train size: {len(train_dataset) }")
print(f"test size: {len(test_dataset)}")

train size: 900
test size: 100


### 把数据集捆成批次
这里要用到dataloader函数,可以指定1批的大小是多大，是否打乱，一般只有训练集需要打乱，测试集不用

In [32]:
from torch.utils.data import DataLoader
#第一个参数传数据集，batch_size指定大小，shuffle指定是否序，num_workers指定进程数，可以加快训练,
train_loader=DataLoader(train_dataset,batch_size=16,shuffle=True,num_workers=0)
test_loader=DataLoader(test_dataset,batch_size=16,shuffle=False,num_workers=0)

在GPU环境下，设置pin_memory=True，可以加快训练，原理跳过。

以上就是最基础的数据处理部分了，在实际训练中往往会复杂多变，但是只要把握好自定义数据集，捆绑的逻辑就可以了

In [34]:
for datas_batch,labels_batch in train_loader:
    #查看一批数据
    for data in zip(datas_batch,labels_batch):
        print(data)
    break

(tensor([0.5075, 0.5707]), tensor(1))
(tensor([0.2954, 0.6791]), tensor(1))
(tensor([0.8238, 0.2523]), tensor(0))
(tensor([0.8660, 0.2553]), tensor(0))
(tensor([0.6617, 0.0438]), tensor(0))
(tensor([0.0682, 0.4964]), tensor(1))
(tensor([0.2097, 0.8135]), tensor(1))
(tensor([0.4283, 0.8257]), tensor(1))
(tensor([0.7680, 0.0088]), tensor(0))
(tensor([0.8102, 0.5122]), tensor(0))
(tensor([0.3818, 0.3241]), tensor(1))
(tensor([0.7965, 0.8039]), tensor(1))
(tensor([0.7986, 0.1838]), tensor(0))
(tensor([0.8160, 0.5937]), tensor(0))
(tensor([0.7581, 0.7989]), tensor(1))
(tensor([0.6741, 0.0443]), tensor(0))


### collate_fn函数

这个函数指定了以什么样的规则去把数据捆起来，虽然我对这个函数用的不多，但是不妨碍它很重要。有时候数据并不是只有输入和标签的，像之前做Embedding时，还要传入offset，而这一步就是在捆绑时完成的,这里写一个简单的捆绑处理函数，体会这个函数的作用。

传入的batch有n个，batch[0]有着第一个数据的输入和标签，batch[0][0]表示第一个数据的输入

In [44]:
def collate_fn(batch):
    datas=[]
    for x in batch:
        data=x[0]
        data[0]+=1#使得所有的x+1
        datas.append(data)

    labels=[x[1] for x in batch]
    return datas,labels
new_train_loader=DataLoader(train_dataset,batch_size=16,shuffle=True,num_workers=0,collate_fn=collate_fn)
for data,label in new_train_loader:
    for _ in zip(data,label):
        print(_)
    break


(tensor([1.0101, 0.2745]), tensor(1))
(tensor([1.7547, 0.7776]), tensor(1))
(tensor([1.9996, 0.0984]), tensor(0))
(tensor([1.3127, 0.5011]), tensor(1))
(tensor([1.0422, 0.3993]), tensor(1))
(tensor([1.2405, 0.1872]), tensor(1))
(tensor([1.2198, 0.2986]), tensor(1))
(tensor([1.7778, 0.2667]), tensor(0))
(tensor([1.7618, 0.7264]), tensor(1))
(tensor([1.2384, 0.9681]), tensor(1))
(tensor([1.7833, 0.2173]), tensor(0))
(tensor([1.6753, 0.7863]), tensor(1))
(tensor([1.8116, 0.7223]), tensor(1))
(tensor([1.1937, 0.4745]), tensor(1))
(tensor([1.4016, 0.3865]), tensor(1))
(tensor([1.9148, 0.1046]), tensor(0))


上面的collate_fn就是负责把所以输入数据的x加上了1，当然在实际应用中，它能做的比这多的多！

# 用内置的数据集
pytorch已经给我们提供了很多内置的数据集，这里以许多计算机视觉中的数据集为例子

In [46]:
import torchvision
import torchvision.transforms as transforms
import os
path="Dataset"
if not os.path.exists(path):
    os.mkdir(path)
dataset=torchvision.datasets.MNIST(root=path,train=True,transform=transforms.ToTensor(),download=True)

100.0%
100.0%
100.0%
100.0%


# 数据预处理和增强
这一部分要用到`torch.transforms`里面的函数，一般可以包括调整大小，转为张量，标准化


In [ ]:
# 定义数据预处理的流水线
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # 将图像调整为 128x128
    transforms.ToTensor(),  # 将图像转换为张量
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
])

### ToTensor
输入一般是PIL图像或者numpy数组，形状需要是HWC类型，使用这个函数后，会把HWC转为CHW，同时把0-255的像素值归一化到0-1内

### Compose
这个函数用来把不同的transforms函数组合，使用时要注意顺序,里面用列表组装

In [ ]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])

如果是对自己的数据集处理的话,可以把transform一起传进去。

In [48]:
class Self_Dataset(Dataset):
    def __init__(self,images_list,lables,transform):
        self.images = images_list
        self.lables = lables
        self.transform = transform
    def __len__(self):
        return len(self.lables)

    def __getitem__(self, item):
        image=self.images[item]
        image_tensor=self.transform(image)
        label=self.labels[item]
        return image_tensor,label